In [12]:
import os
import pydicom
import cv2
import numpy as np

In [13]:
input_dir = 'C:/Users/AKSHAT/Documents/cpcode/dataset/manifest-1682931143289/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192'
output_dir = 'C:/Users/AKSHAT/Documents/cpcode/test/first'

In [14]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from pydicom import dcmread

path = 'C:/Users/AKSHAT/Documents/cpcode/dataset/manifest-1682931143289/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192/1-001.dcm'
print(path)
x = dcmread(path)
print(x)

C:/Users/AKSHAT/Documents/cpcode/dataset/manifest-1682931143289/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192/1-001.dcm
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 206
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.14519.5.2.1.6279.6001.262721256650280657946440242654
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.22213.1.143
(0002, 0013) Implementation Version Name         SH: '0.5'
(0002, 0016) Source Application Entity Title     AE: 'POSDA'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP

In [4]:
pip install -U scikit-image


Note: you may need to restart the kernel to use updated packages.


In [15]:
import skimage
from skimage import measure
from skimage import morphology
from skimage.segmentation import watershed




def lung_segmentation(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        
        if filename.endswith('.dcm'):
            # Loading DICOM image :
            dcm = pydicom.dcmread(os.path.join(input_dir, filename))

            # Converting the DICOM image to OpenCV format :
            img = dcm.pixel_array.astype('uint8')
            
            # Adding zero-padding to the image :
            img_padded = cv2.copyMakeBorder(img, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=0)

            # Applying median filter to the image :
            img_median = cv2.medianBlur(img, 5)

            # Threshold the image to create a binary image :
            _, img_thresh = cv2.threshold(img_median, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

            # Converting binary image to 8-bit unsigned format :
            img_thresh = cv2.convertScaleAbs(img_thresh)

            # Applying distance transform to the binary image :
            dt = cv2.distanceTransform(img_thresh, cv2.DIST_L2, 3)

            # Threshold the distance transform to create markers :
            _, dt_thresh = cv2.threshold(dt, 0.5 * dt.max(), 255, cv2.THRESH_BINARY)

            # Performing morphological closing on the markers :
            dt_thresh = cv2.morphologyEx(dt_thresh, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

            # Performing watershed algorithm to segment the image :
            markers = measure.label(dt_thresh, connectivity=1)
            labels = watershed(-dt, markers, mask=img_thresh)


            # Creating a mask of the lung region :
            lung_mask = np.zeros_like(img_thresh)
            lung_mask[labels > 1] = 255

            # Applying the mask to the original image to extract the lung region :
            lung_img = cv2.bitwise_and(img, lung_mask)

            # Displaying the original image, the binary image, and the segmented lung image :
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))
            axs[0].imshow(img, cmap='gray')
            axs[0].set_title('Original Image')
            axs[1].imshow(img_thresh, cmap='gray')
            axs[1].set_title('Binary Image')
            axs[2].imshow(lung_img, cmap='gray')
            axs[2].set_title('Segmented Lung Image')

            # Saving segmented lung image to output directory :
            output_filename = os.path.splitext(filename)[0] + '.jpg'
            plt.savefig(os.path.join(output_dir, output_filename))
            plt.close()

In [16]:
lung_segmentation(input_dir,output_dir)